## ONLINE RETAIL TRANSACTION ANALYSIS - ETL

Data source: https://www.kaggle.com/datasets/abhishekrp1517/online-retail-transactions-dataset 

## Data analysis goals

Analyse online retail transaction data to understand customer behaviour, identify popular products, and optimise pricing and marketing strategies. Provide insights into customer behaviour, popular products, and pricing strategies to improve sales and marketing efforts.

## Context

The "Online Retail Transaction" dataset contains information on customer transactions made through an online retail platform. It includes data on products purchased, quantities, transaction dates and times, prices, customer identifiers, and customer locations. This dataset can be used to analyse customer behaviour and preferences, identify popular products, and optimise pricing and marketing strategies.

---

# ETL Pipeline (Extract, Transform, Load)

Import Packages

In [2]:
import numpy as np
import pandas as pd

Get current working directory

In [3]:
import os
current_dir = os.getcwd()
current_dir

'c:\\Users\\Joy\\Desktop\\Code Institute\\IFP\\jupyter_notebooks'

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [4]:
os.chdir(os.path.dirname(current_dir))
current_dir = os.getcwd()

In [5]:
input_file_path = current_dir+'\\dataset\\raw\\Online_Retail.csv'
processed_file_path = current_dir+'\\dataset\\processed\\Online_Retail_Processed.csv'
input_file_path

'c:\\Users\\Joy\\Desktop\\Code Institute\\IFP\\dataset\\raw\\Online_Retail.csv'

## E (Extraction): Load data from the provided dataset

Load the dataset

In [6]:
# Load the dataset
df = pd.read_csv(input_file_path)
# Preview the first few rows
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


---

Data Types

In [7]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID       int64
Country         object
dtype: object

---

Basic Statistics

In [8]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,541909.000000
mean,9.552250,4.611114,15287.518434
std,218.081158,96.759853,1484.746041
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,14367.000000
50%,3.000000,2.080000,15287.000000
75%,10.000000,4.130000,16255.000000
max,80995.000000,38970.000000,18287.000000


---

## T(Transform): Clean the data, handle missing values, encode categorical variables, and create new features such as total transaction value

Store original size

In [9]:
original_size = len(df)
original_size

541909

Convert data types

In [10]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['UnitPrice'] = pd.to_numeric(df['UnitPrice'], errors='coerce')
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df['CustomerID'] = pd.to_numeric(df['CustomerID'], errors='coerce')
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID              int64
Country                object
dtype: object

Check if there are missing data

In [11]:
df.isnull().sum()

InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64

Drop rows with missing critical data

In [12]:
# Drop rows with missing data and create a new DataFrame
df_cleaned = df.dropna(subset=['CustomerID', 'InvoiceDate', 'UnitPrice', 'Quantity']).copy()
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]  # Remove returns/cancellations
df_cleaned = df_cleaned[df_cleaned['UnitPrice'] > 0]  # Remove free items
df_cleaned

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France


Remove any duplicates

In [13]:
# Count the number of duplicate rows in the DataFrame
duplicate_count = df_cleaned.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Remove duplicate rows from the DataFrame and create a new DataFrame to avoid SettingWithCopyWarning
df_cleaned = df_cleaned.drop_duplicates().copy()

# Display the DataFrame after removing duplicates
df_cleaned.head()

Number of duplicate rows: 5226


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Cleaned dataset

In [15]:
print(f"Cleaned dataset: {len(df_cleaned):,} records ({original_size - len(df_cleaned):,} removed)")

Cleaned dataset: 524,878 records (17,031 removed)


Creating new features "total transaction value" and time columns

In [16]:
# Create a new column for total transaction
df_cleaned['TotalTransaction'] = (df_cleaned['Quantity'] * df_cleaned['UnitPrice']).round(2)
df_cleaned['Year'] = df_cleaned['InvoiceDate'].dt.year        # Extract the year from InvoiceDate
df_cleaned['Month'] = df_cleaned['InvoiceDate'].dt.month      # Extract the month (as a number) from InvoiceDate
df_cleaned['DayOfWeek'] = df_cleaned['InvoiceDate'].dt.day_name()  # Extract the day of the week (e.g., Monday)
df_cleaned['Hour'] = df_cleaned['InvoiceDate'].dt.hour        # Extract the hour from InvoiceDate
df_cleaned['YearMonth'] = df_cleaned['InvoiceDate'].dt.to_period('M')  # Extract the year and month as a period (e.g., 2021-05)
# Display the cleaned DataFrame
df_cleaned.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalTransaction,Year,Month,DayOfWeek,Hour,YearMonth
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,2010,12,Wednesday,8,2010-12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,12,Wednesday,8,2010-12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,2010,12,Wednesday,8,2010-12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,12,Wednesday,8,2010-12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010,12,Wednesday,8,2010-12


In [17]:
df_cleaned.dtypes

InvoiceNo                   object
StockCode                   object
Description                 object
Quantity                     int64
InvoiceDate         datetime64[ns]
UnitPrice                  float64
CustomerID                   int64
Country                     object
TotalTransaction           float64
Year                         int32
Month                        int32
DayOfWeek                   object
Hour                         int32
YearMonth                period[M]
dtype: object